# Code adapted from
* https://dzlab.github.io/dltips/en/pytorch/torchtext-datasets/ to use custom dataset
* https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/3%20-%20Faster%20Sentiment%20Analysis.ipynb to use model

In [1]:
def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [2]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  preprocessing = generate_bigrams)

LABEL = data.LabelField(dtype = torch.float)

In [3]:
BATCH_SIZE = 128#64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train, val, test = data.TabularDataset.splits(
    path='./data', train='Train.csv',
    validation='Val.csv', test='Test.csv', format='csv',
    fields=[('text', TEXT),(None, None), ('label', LABEL)],
    skip_header = True)# ignore helpfulness column
    # columns are Text, helpfulness, and rating

train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train, val, test), batch_sizes=(BATCH_SIZE,BATCH_SIZE,BATCH_SIZE),
    sort_key=lambda x: len(x.text), device = device)

In [4]:
MAX_VOCAB_SIZE = 50_000

TEXT.build_vocab(train, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train)

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
                
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        return self.fc(pooled)

In [6]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

In [7]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 5,000,301 trainable parameters


In [8]:
pretrained_embeddings = TEXT.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.3456, -0.2499,  0.5868,  ..., -1.3106,  1.0294, -0.0588],
        ...,
        [-0.7159,  0.3680,  0.1050,  ...,  0.5475,  1.1873,  0.6015],
        [-0.1165, -0.2232,  1.3694,  ...,  0.3804, -0.5789,  1.6296],
        [-1.5079, -1.5550, -1.6569,  ...,  1.6023, -0.8018, -1.6895]])

In [9]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [10]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [11]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [12]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [13]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [14]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [15]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [16]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, val_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'sentiment.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 6s
	Train Loss: 0.521 | Train Acc: 81.97%
	 Val. Loss: 0.712 |  Val. Acc: 82.29%
Epoch: 02 | Epoch Time: 1m 50s
	Train Loss: 0.354 | Train Acc: 83.29%
	 Val. Loss: 0.803 |  Val. Acc: 80.07%
Epoch: 03 | Epoch Time: 1m 49s
	Train Loss: 0.291 | Train Acc: 82.63%
	 Val. Loss: 0.887 |  Val. Acc: 79.15%
Epoch: 04 | Epoch Time: 1m 48s
	Train Loss: 0.256 | Train Acc: 82.23%
	 Val. Loss: 0.966 |  Val. Acc: 77.98%
Epoch: 05 | Epoch Time: 1m 47s
	Train Loss: 0.233 | Train Acc: 82.04%
	 Val. Loss: 1.015 |  Val. Acc: 78.39%
Epoch: 06 | Epoch Time: 1m 49s
	Train Loss: 0.213 | Train Acc: 81.89%
	 Val. Loss: 1.068 |  Val. Acc: 78.54%
Epoch: 07 | Epoch Time: 1m 55s
	Train Loss: 0.196 | Train Acc: 81.81%
	 Val. Loss: 1.124 |  Val. Acc: 78.60%
Epoch: 08 | Epoch Time: 1m 48s
	Train Loss: 0.181 | Train Acc: 81.70%
	 Val. Loss: 1.198 |  Val. Acc: 77.88%
Epoch: 09 | Epoch Time: 1m 58s
	Train Loss: 0.167 | Train Acc: 81.56%
	 Val. Loss: 1.292 |  Val. Acc: 77.03%
Epoch: 10 | Epoch Ti

In [17]:
model.load_state_dict(torch.load('sentiment.pt'))

test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.721 | Test Acc: 82.14%


In [18]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = generate_bigrams([tok.text for tok in nlp.tokenizer(sentence)])
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [19]:
predict_sentiment(model, "This movie SUCKS HARD")

0.324064165353775

In [20]:
predict_sentiment(model, "This book is FANTASTIC")

0.999984622001648

# Problems
- Some words are not processed correctly: good has high sentiment but fantastic does not

In [21]:
import pandas as pd
df= pd.read_csv("Tokenized.csv")
df.head()

,Text,Helpfulness,Rating
0,"['daughter', 'wanted', 'book', 'price', 'Amazo...",1.000000,1
1,"['bought', 'zoku', 'quick', 'pop', 'daughterr'...",1.000000,1
2,"['shortage', 'pop', 'recipes', 'available', 'f...",2.059259,1
3,"['book', 'must', 'get', 'Zoku', 'also', 'highl...",1.855556,1
4,"['cookbook', 'great', 'really', 'enjoyed', 're...",1.000000,1


In [22]:
df["Prediction"]= df["Text"].map(lambda x:predict_sentiment(model, x))

In [23]:
df["Scaled_Prediction"]= df.apply(lambda row:(row['Prediction']*row['Helpfulness']), axis=1)

In [24]:
df.head()

,Text,Helpfulness,Rating,Prediction,Scaled_Prediction
0,"['daughter', 'wanted', 'book', 'price', 'Amazo...",1.000000,1,9.475932e-10,9.475932e-10
1,"['bought', 'zoku', 'quick', 'pop', 'daughterr'...",1.000000,1,5.314376e-12,5.314376e-12
2,"['shortage', 'pop', 'recipes', 'available', 'f...",2.059259,1,7.398023e-03,1.523445e-02
3,"['book', 'must', 'get', 'Zoku', 'also', 'highl...",1.855556,1,5.322372e-11,9.875956e-11
4,"['cookbook', 'great', 'really', 'enjoyed', 're...",1.000000,1,8.707701e-07,8.707701e-07


In [25]:
df.to_csv("results.csv",index=False)

In [29]:
df.Rating.value_counts()

 1    455204
-1     51419
 0     45059
Name: Rating, dtype: int64